In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [ ]:
import json
with open('../KG-FPQ-data/art_YN.json', 'r') as file:
    json_data = json.load(file)

T_q = []
F_q = []
for data in json_data:
    T_q.append(data['TPQ'])
    for key, value in data.items():
        if key.startswith("FPQ_"):
            F_q.append(value)

all_q = T_q + F_q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    # "Qwen/Qwen2.5-7B-Instruct",
    "Qwen/Qwen1.5-7B-Chat",
    torch_dtype="auto",
    device_map="auto"
)
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B-Chat")

### direct RAG

In [ ]:
p = """Given the context below, does the following question contain a false premise? Answer with 'Yes' or 'No' only. Note that the context is provided
as valid facts in a triple."""
prompt = """
Context: {context}
Query: {query}
"""

with open("../max_cos_sim_dict.json", "r") as file:
    max_lst = json.load(file)

responses = []
for i in range(len(all_q)):
    qq = prompt.format(context=max_lst[i]['graph_triplet'], query=all_q[i])
    messages = [
        {"role": "system", "content": p},
        {"role": "user", "content": qq}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        attention_mask=model_inputs.attention_mask,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)

### fasle premise detection

In [ ]:

responses = []
for i in range(len(all_q)):
    messages = [
        {"role": "system", "content": "Does the following question contain a false premise? Answer with 'Yes' or 'No' only."},
        {"role": "user", "content": all_q[i]}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        attention_mask=model_inputs.attention_mask,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)

In [ ]:
p = """Given the context below, does the following question contain a false premise? Answer with 'Yes' or 'No' only. Note that the context is provided
as valid facts in a triple."""
prompt = """
Context: {context}
Query: {query}
"""

with open("../max_cos_sim_dict.json", "r") as file:
    max_lst = json.load(file)

responses = []
for i in range(len(all_q)):
    qq = prompt.format(context=max_lst[i]['graph_triplet'], query=all_q[i])
    messages = [
        {"role": "system", "content": p},
        {"role": "user", "content": qq}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        attention_mask=model_inputs.attention_mask,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)

In [ ]:
with open("g_retri_g.json", "r") as file:
    sub_g = json.load(file)
unique_l = []
for i in list(sub_g):
    x = list(map(list, set(map(tuple, i))))
    if x ==[]:
        unique_l.append("")
    else:
        unique_l.append(x[0])
unique_l

In [ ]:
p = """Given the context below, does the following question contain a false premise? Answer with 'Yes' or 'No' only. Note that the context is provided
as valid facts in a triple."""

prompt = """
Context: {context}
Query: {query}
Answer:"""

responses = []
for i in range(len(all_q)):
    qq = prompt.format(context=unique_l[i], query=all_q[i])
    messages = [
        {"role": "system", "content": p},
        {"role": "user", "content": qq}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        attention_mask=model_inputs.attention_mask,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)

In [ ]:
for i in range(len(all_q)):
    print(unique_l[i], all_q[i])

### end

In [ ]:
# prompt = "Give me a short introduction to large language model."
responses = []
for q in all_q:
    messages = [
        # {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": q}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)


In [ ]:
# with open('qwen_orig_ans.json', 'w') as file:
#     json.dump(responses, file)

In [ ]:
#prompt
responses = []
for q in all_q:
    qq = q + ' This question may contain a fasle premise.'
    messages = [
        # {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": qq}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        # do_sample=False,
        # top_p=1,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append(response)

# with open('qwen_prompt_ans.json', 'w') as file:
#     json.dump(responses, file)


print('QWEN prompt ANS ver2')

# with open('hal_orig_ans.json', 'r') as file:
#     responses = json.load(file)

In [ ]:
#prompt
responses = []
for q in all_q:
    vote_y = 0
    vote_n = 0
    for i in range(3):
        messages = [
            # {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": q}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            # do_sample=False,
            # top_p=1,  
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        r = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        if r[0] == 'Y':
            vote_y+=1
        if r[0] == 'N' or ('no' in r.lower() and r[0] != 'Y'):
            vote_n+=1
    
    if vote_y > vote_n:
        responses.append('Yes')
    if vote_y < vote_n:
        responses.append('No')

# with open('qwen_majority.json', 'w') as file:
#     json.dump(responses, file)


print('QWEN majority vote')

# with open('hal_orig_ans.json', 'r') as file:
#     responses = json.load(file)